# Function to Create the Dataset

In [1]:
def DFCreator (df,fin):
    #libraries
    import numpy as np
    import scipy.stats
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    from statsmodels.tsa.stattools import kpss
    from statsmodels.tsa.stattools import adfuller
    from statsmodels.tsa.api import Holt
    from arch import arch_model
    from arch.univariate.base import ARCHModelResult
    #format raw df
    def totime(df):
        """Convierte la columna que se llama ts a tipo DateTime y elimina los extras(+2:00)"""
        df[time_col]= df[time_col].map(lambda x: str(x)[:-6])
        df[time_col] = pd.to_datetime(df[time_col], format='%Y-%m-%d %H:%M:%S')
        
        return df
    #functions that output df with new features
    def basic(df,name,new):
        """Basic stats"""
        try:
            new['min']= df[name].min()
            new['max']= df[name].max()
            new['mean']= df[name].mean()
            new['median']= df[name].median()
            new['range']= new['max']-new['min']
            new['variance']= np.var(df[name])
        except:
            new['min']= np.nan
            new['max']= np.nan
            new['mean']= np.nan
            new['median']= np.nan
            new['range']= np.nan
            new['variance']= np.nan
        return new
    
    def windows (df, name, new):
        """lumpiness, stability, max_level_shift, max_var_shift,flat_spots"""
        try:
            #get how many datapoints we have per month
            length = round(len(df)/12)
            #divide in 12 windows
            window1= df[name][:length-1]
            window2=df[name][length:(length*2)-1]
            window3=df[name][length*2:(length*3)-1]
            window4=df[name][length*3:(length*4)-1]
            window5=df[name][length*4:(length*5)-1]
            window6=df[name][length*5:(length*6)-1]
            window7=df[name][length*6:(length*7)-1]
            window8=df[name][length*7:(length*8)-1]
            window9=df[name][length*8:(length*9)-1]
            window10=df[name][length*9:(length*10)-1]
            window11=df[name][length*10:(length*11)-1]
            window12=df[name][length*11:]
            l = [window1,window2,window3,window4,window5,window6,window7,window8,window9,window10,window11,window12]
            #stability and lumpiness
            var= []
            me= []
            for i in l:
                v= np.var(i)
                var.append(v)
                m= np.mean(i)
                me.append(m)
            #max_level_shift
            shifts =[]
            for j in range(len(me)-1):
                sh = me[j]-me[j+1]
                shifts.append(sh)
            #max_var_shift
            shifts2 =[]
            for j in range(len(var)-1):
                sh2 = var[j]-var[j+1]
                shifts2.append(sh2)
            #flat_spots
            def countlist(random_list):
                retlist = []
                count=1
                # Avoid IndexError for  random_list[i+1]
                for i in range(len(random_list) - 1):
                    # Check if the next number is consecutive
                    if (random_list.loc[i] == random_list.loc[i+1]).all():
                        count += 1
                    else:
                        # If it is not append the count and restart counting
                        retlist.append(count)
                        count = 1
                # Since we stopped the loop one early append the last count
                retlist.append(count)
                return retlist

            window2=window2.reset_index()
            window2.drop('index',axis=1,inplace=True)
            window3=window3.reset_index()
            window3.drop('index',axis=1,inplace=True)
            window4=window4.reset_index()
            window4.drop('index',axis=1,inplace=True)
            window5=window5.reset_index()
            window5.drop('index',axis=1,inplace=True)
            window6=window6.reset_index()
            window6.drop('index',axis=1,inplace=True)
            window7=window7.reset_index()
            window7.drop('index',axis=1,inplace=True)
            window8=window8.reset_index()
            window8.drop('index',axis=1,inplace=True)
            window9=window9.reset_index()
            window9.drop('index',axis=1,inplace=True)
            window10=window10.reset_index()
            window10.drop('index',axis=1,inplace=True)
            l = [window1,window2,window3,window4,window5,window6,window7,window8,window9,window10]
            for i in l:
                rle=[]
                retlist=countlist(i)
                #append max run length of each window
                mxw= max(retlist)
                rle.append(mxw)
            #calculate max rle between all windows
            new['flat_spots']= max(rle)
            new['max_var_shift']= max(shifts2)
            new['max_level_shift']= max(shifts)
            new['lumpiness']= np.var(var)
            new['stability']= np.var(me)
        except:
            new['flat_spots']= np.nan
            new['max_var_shift']= np.nan
            new['max_level_shift']= np.nan
            new['lumpiness']= np.nan
            new['stability']=np.nan
        return new
    
    def entropy(df, name, new):
        """estimate of the spectral density of the data. This measures the “forecastability” of a time series, where low values indicate a high signal-to-noise ratio, 
        and large values occur when a series is difficult to forecast."""
        try:
            counts = df[name].value_counts()
            new['entropy'] = scipy.stats.entropy(counts)
        except:
            new['entropy'] = np.nan
        return new
    
    def unitroot_kpss(df,name, new):
        """computes kpss from 1 lag"""
        try:
            kp = kpss(df[name], nlags=1)
            new['KPSS']= kp[0]
            new['KPSS_pValue']= kp[1]
        except:
            new['KPSS']= np.nan
            new['KPSS_pValue']= np.nan
        return new
    
    def abs_energy(df,name, new):
        """sum of the squared values"""
        try:
            new['squared']= df[name]**2
            new['abs_energy']=new.squared.sum()
            new.drop('squared',axis=1,inplace=True)
        except:
            new['abs_energy']=np.nan
        return new
    
    def absolute_sum_of_changes(df,name, new):
        try:
            changes=[]
            for i in range(len(df)-1):
                ch= abs(df[name][i]-df[name][i+1])
                changes.append(ch)
            new['absolute_sum_of_changes']= sum(changes)
        except:
            new['absolute_sum_of_changes']= np.nan
        return new
    
    def adf_test(df,name,new):
        """returns adf value and p-value"""
        try:
            adf = adfuller(df[name], autolag='AIC')
            new['ADF_Value']= adf[0]
            new['ADF_pValue']= adf[1]
        except:
            new['ADF_Value']= np.nan
            new['ADF_pValue']= np.nan
        return new
    
    def count_above_mean(df,name, new):
        """how many values are above the mean"""
        try:
            m= df[name].mean()
            count=0
            for i in df[name]:
                if i> m:
                    count=count +1
                else:
                    count=count
            new['count_above_mean']= count
        except:
            new['count_above_mean']= np.nan
        return new
    
    def count_below_mean(df,name,new):
        """how many values are below the mean"""
        try:
            m= df[name].mean()
            count=0
            for i in df[name]:
                if i< m:
                    count=count +1
                else:
                    count=count
            new['count_below_mean']= count
        except:
            new['count_below_mean']= np.nan
        return new
    
    def kurtosis(df,name,new):
        try:
            new['kurtosis']= scipy.stats.kurtosis(df[name])
        except:
            new['kurtosis']= np.nan
        return new
    
    def mean_absolute_change(df,name,new):
        try:
            changes=[]
            for i in range(len(df)-1):
                ch= abs(df[name][i]-df[name][i+1])
                changes.append(ch)
            new['mean_absolute_change']= np.mean(changes)
        except:
            new['mean_absolute_change']= np.nan
        return new
    
    def skewness (df, name,new):
        try:
            new['skewness']= scipy.stats.skew(df[name])
        except:
            new['skewness']= np.nan
        return new
    
    def crossing_points(df,name,new):
        """how many times the median line is crossed"""
        try:
            m= df[name].median()
            count=0
            for i in range(len(df[name])-1):
                if df[name][i]<= m and df[name][i+1]>= m:
                    count +=1
                elif df[name][i]>= m and df[name][i+1]<= m:
                    count +=1
            new['crossing_points']= count
        except:
            new['crossing_points']= np.nan
        return new
    
    def holt(df,name,new):
        """returns alpha(soothing parameter for level) and beta(soothing parameter for trend) of holt model"""
        try:
            fit2 = Holt(df[name], exponential=True).fit()
            results=pd.DataFrame()
            results['names']=['alpha',"beta","phi","gamma","l0","b0","SSE"]
            params = ['smoothing_level', 'smoothing_slope', 'damping_slope', 'smoothing_seasonal', 'initial_level', 'initial_slope']
            results["Holt"]       = [fit2.params[p] for p in params] + [fit2.sse]
            new['holt_alpha']= results['Holt'][0]
            new['holt_beta']= results['Holt'][1]
        except:
            new['holt_alpha']= np.nan
            new['holt_beta']= np.nan
        return new
    
    def hw(df,name,new):
        """returns parameters of holt winters (level, trend and season)"""
        try:
            fit2 = ExponentialSmoothing(df[name]).fit()
            results = pd.DataFrame(np.c_[df[name], fit2.level, fit2.slope, fit2.season, fit2.fittedvalues],
                          columns=['yt','lt','bt','st','yhat'],index=df[name].index)
            new['hw_level']= results['lt'].mean()
            new['hw_slope']= results['bt'].mean()
            new['hw_season']= results['st'].mean()
        except:
            new['hw_level']= np.nan
            new['hw_slope']= np.nan
            new['hw_season']= np.nan
        return new
    
    def heteroscedasticity(df,name,new):
        
        try:
            model = arch_model(df[name], mean='Zero', vol='GARCH', p=1, q=1)
            res= model.fit()

            lm=res.arch_lm_test()
            lms = str(lm)
            new['lm_Statistic']=lms[119:129]
            new['lm_pValue']=lms[139:-22]
        except:
            new['lm_Statistic']=np.nan
            new['lm_pValue']=np.nan
        return new
    
    
    time_col=df.columns[0]
    name=df.columns[1]
    df=totime(df)
    

    data = {'id':  [name]}
    new = pd.DataFrame (data, columns = ['id'])
    new=basic(df,name,new)
    new= windows(df,name,new)
    
    new= entropy(df,name,new)
    
    new= unitroot_kpss(df,name,new)
    new= abs_energy(df,name,new)
    new= absolute_sum_of_changes(df,name,new)
    new= adf_test(df,name,new)
    new= count_above_mean(df,name,new)
    new= count_below_mean(df,name,new)
    new= kurtosis(df,name,new)
    new= mean_absolute_change(df,name,new)
    new= skewness (df, name,new)
    new= crossing_points(df,name,new)
    new=  holt(df,name,new)
    new= hw(df,name,new)
    new=heteroscedasticity(df,name,new)
    fin=fin.append(new)
    return fin
    

In [2]:
import pandas as pd

## Creating a new Dataframe to append the new data

In [3]:
fin=pd.DataFrame()

In [4]:
fin.head()

""


## Creating a list with all CSVs of that path

In [5]:
import glob
c=glob.glob("./DONE/*.csv")

## Creating the Dataset 

In [6]:
for i in c:
    df = pd.read_csv(i)
    fin= DFCreator (df,fin)

/Users/rocio/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


Iteration:      1,   Func. Count:      5,   Neg. LLF: 61662.001870029824
Iteration:      2,   Func. Count:     13,   Neg. LLF: 61661.781683578636
Iteration:      3,   Func. Count:     18,   Neg. LLF: 61656.60412939067
Iteration:      4,   Func. Count:     23,   Neg. LLF: 61656.59114434909
Iteration:      5,   Func. Count:     28,   Neg. LLF: 61656.53546818458
Iteration:      6,   Func. Count:     33,   Neg. LLF: 61656.437165223266
Iteration:      7,   Func. Count:     38,   Neg. LLF: 61656.43556068744
Iteration:      8,   Func. Count:     43,   Neg. LLF: 61656.42753392954
Iteration:      9,   Func. Count:     48,   Neg. LLF: 61656.401162457376
Iteration:     10,   Func. Count:     53,   Neg. LLF: 61656.39167780213
Iteration:     11,   Func. Count:     59,   Neg. LLF: 61656.39167545806
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 61656.391675455634
            Iterations: 11
            Function evaluations: 70
            Gradient evaluatio

/Users/rocio/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


Iteration:      1,   Func. Count:      5,   Neg. LLF: 146681.04059945856
Iteration:      2,   Func. Count:     13,   Neg. LLF: 146680.46073189742
Iteration:      3,   Func. Count:     19,   Neg. LLF: 146671.94013193998
Iteration:      4,   Func. Count:     24,   Neg. LLF: 146671.1570976472
Iteration:      5,   Func. Count:     29,   Neg. LLF: 146670.1522144177
Iteration:      6,   Func. Count:     34,   Neg. LLF: 146669.9446666834
Iteration:      7,   Func. Count:     39,   Neg. LLF: 146669.90304189367
Iteration:      8,   Func. Count:     44,   Neg. LLF: 146669.90026305188
Iteration:      9,   Func. Count:     49,   Neg. LLF: 146669.90014923053
Iteration:     10,   Func. Count:     55,   Neg. LLF: 146669.9001439063
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 146669.9001439165
            Iterations: 10
            Function evaluations: 66
            Gradient evaluations: 10


/Users/rocio/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


...

Iteration:      1,   Func. Count:      5,   Neg. LLF: 157085.28556649273
Iteration:      2,   Func. Count:     13,   Neg. LLF: 157084.36958138982
Iteration:      3,   Func. Count:     19,   Neg. LLF: 157022.55908236554
Iteration:      4,   Func. Count:     25,   Neg. LLF: 156997.05550925332
Iteration:      5,   Func. Count:     31,   Neg. LLF: 156973.5155395509
Iteration:      6,   Func. Count:     37,   Neg. LLF: 156951.72618377
Iteration:      7,   Func. Count:     43,   Neg. LLF: 156915.4815548929
Iteration:      8,   Func. Count:     50,   Neg. LLF: 156911.6744652297
Iteration:      9,   Func. Count:     56,   Neg. LLF: 156882.74121078415
Iteration:     10,   Func. Count:     62,   Neg. LLF: 156865.1378825504
Iteration:     11,   Func. Count:     68,   Neg. LLF: 156848.77421558686
Iteration:     12,   Func. Count:     74,   Neg. LLF: 156834.61430213088
Iteration:     13,   Func. Count:     80,   Neg. LLF: 156821.91303656017
Iteration:     14,   Func. Count:     86,   Neg. LLF: 1568

/Users/rocio/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


Iteration:      1,   Func. Count:      5,   Neg. LLF: 158478.42569072032
Iteration:      2,   Func. Count:     13,   Neg. LLF: 158478.27937407017
Iteration:      3,   Func. Count:     19,   Neg. LLF: 158466.83434390076
Iteration:      4,   Func. Count:     24,   Neg. LLF: 158464.75346322832
Iteration:      5,   Func. Count:     29,   Neg. LLF: 158463.74669319845
Iteration:      6,   Func. Count:     35,   Neg. LLF: 158462.64084403173
Iteration:      7,   Func. Count:     40,   Neg. LLF: 158462.3822034839
Iteration:      8,   Func. Count:     45,   Neg. LLF: 158462.07140035817
Iteration:      9,   Func. Count:     50,   Neg. LLF: 158461.91058960577
Iteration:     10,   Func. Count:     55,   Neg. LLF: 158461.8708059225
Iteration:     11,   Func. Count:     60,   Neg. LLF: 158461.86681616382
Iteration:     12,   Func. Count:     65,   Neg. LLF: 158461.86670175235
Iteration:     13,   Func. Count:     70,   Neg. LLF: 158461.8666661097
Iteration:     14,   Func. Count:     75,   Neg. LLF: 

/Users/rocio/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


Iteration:      1,   Func. Count:      5,   Neg. LLF: 153326.80991965326
Iteration:      2,   Func. Count:     11,   Neg. LLF: 152576.58919039433
Iteration:      3,   Func. Count:     18,   Neg. LLF: 152480.73771157296
Iteration:      4,   Func. Count:     24,   Neg. LLF: 151904.94240131148
Iteration:      5,   Func. Count:     30,   Neg. LLF: 151519.16195998417
Iteration:      6,   Func. Count:     36,   Neg. LLF: 150625.02667763337
Iteration:      7,   Func. Count:     41,   Neg. LLF: 150310.32821572298
Iteration:      8,   Func. Count:     47,   Neg. LLF: 149999.7588379362
Iteration:      9,   Func. Count:     53,   Neg. LLF: 149710.6560356911
Iteration:     10,   Func. Count:     58,   Neg. LLF: 146565.7007283652
Iteration:     11,   Func. Count:     64,   Neg. LLF: 146398.8440787373
Iteration:     12,   Func. Count:     70,   Neg. LLF: 146237.5263925505
Iteration:     13,   Func. Count:     76,   Neg. LLF: 146082.35864832296
Iteration:     14,   Func. Count:     82,   Neg. LLF: 14

In [9]:
fin.shape

(1608, 32)

In [10]:
fin.head()

,id,min,max,mean,median,range,variance,flat_spots,max_var_shift,max_level_shift,...,mean_absolute_change,skewness,crossing_points,holt_alpha,holt_beta,hw_level,hw_slope,hw_season,lm_Statistic,lm_pValue
0,-M_IsColbrand_J2-$28024$29$20P2N_RC_24_TempAmb,14.1800,31.600000,22.891772,22.670000,17.420000,12.344444,8,7.710445,3.270672,...,0.064385,0.249736,136,0.939930,0.329398,22.891772,0.0,0.0,13581.3965,0.0000
0,-CC_P2_OcF_2-FCIT_OcF_2_CC_OcF_01_TA,0.0000,24.624001,17.752727,17.625601,24.624001,7.571681,11,19.640775,4.256843,...,0.086940,0.273237,744,NaN,NaN,17.752727,0.0,0.0,34128.7403,0.0000
0,-CC_PB_NoB-FC_NoB_0_CC_NoB_02_TA,0.0000,29.813101,23.276894,23.292900,29.813101,6.935380,5,0.821676,2.191289,...,0.058445,-0.159094,250,NaN,NaN,23.276894,0.0,0.0,33420.0407,0.0000
0,-CC_P11_LV-FCIT_Lv_12_CC_Lv11_03_TA,12.9525,24.232901,20.760238,21.594500,11.280400,2.935851,5,2.281562,2.416778,...,0.169795,-1.255697,3644,0.305154,0.173398,20.760144,0.0,0.0,33801.6518,0.0000
0,-Recoletos_7-P03_TemAmb2,15.4000,31.700001,23.572855,23.799999,16.300001,8.743074,44,3.457851,2.705444,...,0.083035,0.015828,1073,1.000000,0.154152,23.572855,0.0,0.0,30979.7455,0.0000


In [7]:
fin.to_csv('FINAL.csv')

Rocío González Lantero